# The Knapsack Problem

You can run a cell either by clicking `Run` on the toolbar or by pressing `CTRL+RETURN`. Pressing `SHIFT+RETURN` will run the cell and move the pointer to the next one. For a complete list of keyboard shortcuts press `H`.

## Introduction

In this Jupyter notebook we are going to model the 0-1 knapsack problem together. We will use visualizations extencively to make sure that you can easily follow the process. We encourage you to also use visualizations when you are working alone.  
Now, let us define the problem.

We are given a knapsack with capacity $b$ as well as $n$ items of size $a_i$ and profit $p_i, i \in \{1, \ldots ,n\}$.  
Our task is to *maximize* the total profit of the items that we decide to pack into the knapsack while respecting its capacity.

Run the cell below to import all the necessary libraries. In this case we import `Gurobi` and `matplotlib`. The second library will be used for plotting a couple of graphs later.

In [ ]:
from gurobipy import *
import matplotlib.pyplot as plt

# Enable high resolution plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

## Data

Below you will find the data supplied by your professor/manager/friend/etc.  
The following data is available to you:  
`b`: the capacity of the knapsack  
`n`: the number of items  
`a`: a list containing the sizes of the items  
`p`: a list containing the profits of the items  
`items`: for your convenience, a list of item "names"; in our case $\{0,\ldots,n-1\}$

In [ ]:
b = 40
a = [3, 6, 3, 2, 3, 3, 9, 12, 5, 3, 8, 1]
p = [4, 5, 3, 5, 5, 1, 12, 20, 9, 3, 10, -1]
assert len(a) == len(p), "The lists a and p need to have equal length"
n = len(a)
items = range(n)

print(f'We are given {n} items. The capacity of the knapsack is {b}.')

## Model

We start off by creating an empty model `m`.<br>
Reference: [Model()](https://www.gurobi.com/documentation/9.0/refman/py_model2.html)

In [ ]:
m = Model('knapsack')

Then we add the necessary variables.  
In our case, we will need $n$ binary variables. The variables, let us call them $x_i$, will depict whether the item $i\in \{0, \ldots, n-1\}$ is packed or not.  
Reference: [addVars()](https://www.gurobi.com/documentation/9.0/refman/py_model_addvars.html)

In [ ]:
x = m.addVars(n, vtype=GRB.BINARY, name='x')

Now, before we can start constructing our model, we need to formulate the objective function and the constraints "on paper".  
We want to maximize the total value of the items in the knapsack. Hence, our objective function can be written as
$$max \sum_{i=1}^np_ix_i$$
We will use the efficient function `quicksum()` provided in the `Gurobi` library.<br>
Reference: [setObjective()](https://www.gurobi.com/documentation/9.0/refman/py_model_setobjective.html), [quicksum()](https://www.gurobi.com/documentation/9.0/refman/py_quicksum.html)

In [ ]:
m.setObjective(quicksum(p[i] * x[i] for i in items), GRB.MAXIMIZE)

We'll run the model once without adding any constraints to see what happens.  
Reference: [update()](https://www.gurobi.com/documentation/9.0/refman/py_model_update.html), [optimize()](https://www.gurobi.com/documentation/9.0/refman/py_model_optimize.html)

In [ ]:
m.update()
m.optimize()

Using a bit of Python magic, we can print out the optimal objective value and packed items.<br/>
Reference: [ObjVal](https://www.gurobi.com/documentation/9.0/refman/objval.html), [Variable Attributes](https://www.gurobi.com/documentation/9.0/refman/variable_attributes.html)

In [ ]:
print('The total profit is', round(m.ObjVal, 2))
packed_items = [int(v.varName[2:-1]) for v in m.getVars() if v.x > .5]
print('We packed the items', ', '.join([str(i) for i in packed_items]))

As expected, all items have been packed into the knapsack except for the last one, since packing the last item would decrease the total profit.  
We can visualize the current optimal solution. We will define a helper function `draw_knapsack()`, which we are going to reuse later. This function will draw the current contents of the knapsack for us. The packed items will be labeled and the knapsack capacity will be marked red.

In [ ]:
def draw_knapsack(title=None):
    plt.figure(figsize=(12, 2))
    plt.plot((b, b), (0, 1), lw=2, color='r')
    current_size = 0
    for i in packed_items:    
        plt.bar(current_size+a[i]/2, height=1, width=a[i], edgecolor='k', color='#CAEFD1')
        plt.text(current_size+a[i]/2, .45, i, ha='center', va='bottom')
        current_size += a[i]
    plt.yticks([])
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.xlabel('Total size')
    if title:
        plt.title(title)
    plt.show()

Now we call `draw_knapsack()` and see, that our knapsack is overpacked. Obviously, this is the case, because we haven't yet added the capacity constraint to our model.

In [ ]:
draw_knapsack('Knapsack without capacity constraint')

The capacity constraint may be written as
$$\sum_{i=1}^n a_i x_i \leq b$$
Let us add this constraint to our model. We will once again use the `quicksum()` method.  
Reference: [addConstr()](https://www.gurobi.com/documentation/9.0/refman/py_model_addconstr.html), [quicksum()](https://www.gurobi.com/documentation/9.0/refman/py_quicksum.html)

In [ ]:
m.addConstr(quicksum(a[i] * x[i] for i in items) <= b)

Update the model to process the pending modifications (added variables and constraints) and run the optimization.<br/>
Reference: [update()](https://www.gurobi.com/documentation/9.0/refman/py_model_update.html), [optimize()](https://www.gurobi.com/documentation/9.0/refman/py_model_optimize.html)

In [ ]:
m.update()
m.optimize()

Let us retrieve the new optimal objective value as well as the packed items. Note that fewer items have been packed and the optimum is lower.<br/>
Reference: [ObjVal](https://www.gurobi.com/documentation/9.0/refman/objval.html), [Variable Attributes](https://www.gurobi.com/documentation/9.0/refman/variable_attributes.html)

In [ ]:
print('The total profit is', round(m.ObjVal, 2))
packed_items = [int(v.varName[2:-1]) for v in m.getVars() if v.x > .5]
print('We packed the items', ', '.join([str(i) for i in packed_items]))

Take a look at the knapsack.

In [ ]:
draw_knapsack('Knapsack with capacity constraint')

We can also manually check the solution (which we usually don't have to).

In [ ]:
for i in packed_items:
    print(f'Item {i} | Size: {a[i]} | Profit: {p[i]}')

total_size = sum(a[i] for i in packed_items)
total_profit = sum(p[i] for i in packed_items)
assert total_size <= b, 'The knapsack is overpacked'
assert total_profit == m.ObjVal, 'The objective value is wrong'

print('Total size:', total_size)
print('Total profit:', total_profit)

We see (both visually and arithmetically) that all packed items indeed fit into the knapsack, and the objective value is correct. Yay!

## Complications

Now, let us add a couple more constraints and see how the contents of the knapsack change.

*Additional constraints*: Say, we do not want to bother with small items anymore and decide to only pack items that are larger than 3. We can implement these constraints by explicitly setting the variable $x_i$ to 0 for all items $i \in \{0, \ldots, n-1\}$ that have a size of at most 3.  
Reference: [addConstrs()](https://www.gurobi.com/documentation/9.0/refman/py_model_addconstrs.html)

In [ ]:
m.addConstrs(x[i] == 0 for i in items if a[i] <= 3)

In [ ]:
m.update()
m.optimize()

In [ ]:
print('The total profit value is', round(m.ObjVal, 2))
packed_items = [int(v.varName[2:-1]) for v in m.getVars() if v.x > .5]
print('We packed the items', ', '.join([str(i) for i in packed_items]))
draw_knapsack()

Note that the two smaller items 3 and 4 have been replaced by one larger (actually larger than items 3 and 4 combined) but less valuable item 1. The total profit has dropped accordingly.

We will add two more constraints.  
*Additional constraints*: The items 6 and 10 may not be packed at the same time. Also, the item 1 may only be packed if the item 2 is present.

In [ ]:
m.addConstr(x[6] + x[10] <= 1)
m.addConstr(x[2] >= x[1])

In [ ]:
m.update()
m.optimize()

In [ ]:
print('The total profit is', round(m.ObjVal, 2))
packed_items = [int(v.varName[2:-1]) for v in m.getVars() if v.x > .5]
print('We packed the items', ', '.join([str(i) for i in packed_items]))
draw_knapsack()

Both items 1 and 10 are no longer in the knapsack. Why?  

The item 1 has been "taken out" because, according to the recently added constraint (the second one), it may only be packed together with the item 2. The item 2 is not larger than 3, therefore it may not be packed (remember the constraints added previously). Hence, the item 1 may not be packed as well. 

The item 10 has not been packed because of the first constraint that we have added in the last step: either the item 6 or the item 10 may be packed, but not both. Since the item 6 (profit 9) is more valuable than the item 10 (profit 8), the item 6 stayed in the knapsack, while the item 10 has been removed.

One question you may be asking yourself: "Why weren't any new items added to the knapsack? There is plenty of space..."  
Indeed, why?  
Because all other items are not larger than 3, or, as it is the case with the item 11 which has negative profit, are detrimental to our objective.

## Conclusion

In this notebook you have modeled the 0-1 knapsack problem in its classical formulation, as well as added a couple constraints that further restricted the feasible region.  
You have also used visualization tools to better understand the effects of added constraints.

We hope you've learned something today. See you next time!